In [1]:
import tensorflow as tf
import torch
import torch.nn as nn
import torch.nn.functional as F

/usr/local/anaconda2/envs/pt-tf-env/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/usr/local/anaconda2/envs/pt-tf-env/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/local/anaconda2/envs/pt-tf-env/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


### 分类问题
- 分类问题是是机器学习中最常见的问题之一了。如CV领域识别一张图片中的动物是猫、狗还是鸡；在NLP领域预测下一个最有可能出现的词。
- $cross\_entropy=-\sum_{k=1}^{N}\left(p_{k} * \log q_{k}\right)$
```
torch.nn	torch.nn.functional (F)
CrossEntropyLoss	cross_entropy
LogSoftmax	log_softmax
NLLLoss	nll_loss
```
- softmax回归的输出值个数等于标签里的类别数。因为一共有4种特征和3种输出动物类别。
- 既然分类问题需要得到离散的预测输出，一个简单的办法是将输出值oi当作预测类别是i的置信度，并将值最大的输出所对应的类作为预测输出，即输出 $argmax_i{O_i}$ 。例如，如果o1，o2，o3, 分别为0.1, 10, 0.1，由于o2最大，那么预测类别为2，其代表猫。
- 直接使用输出层的输出有两个问题。一方面，由于输出层的输出值的范围不确定，我们难以直观上判断这些值的意义。例如，刚才举的例子中的输出值10表示“很置信”图像类别为猫，因为该输出值是其他两类的输出值的100倍。但如果o1=03=1000, 那么输出值10却又表示图像类别为猫的概率很低。另一方面，由于真实标签是离散值，这些离散值与不确定范围的输出值之间的误差难以衡量。

In [17]:
y = torch.tensor([[0.0, 0.0, 1.0], [1.0, 0.0, 0.0]], dtype=torch.int64)
y

tensor([[0, 0, 1],
        [1, 0, 0]])

In [21]:
x =  torch.tensor([[2.0, -1.0, 3.0], [1.0, 0.0, -0.5]])
x

tensor([[ 2.0000, -1.0000,  3.0000],
        [ 1.0000,  0.0000, -0.5000]])

In [22]:
?nn.CrossEntropyLoss

In [23]:
F.cross_entropy(x, y)

RuntimeError: multi-target not supported at ../aten/src/THNN/generic/ClassNLLCriterion.c:20

### sparse_softmax_cross_entropy_with_logits

In [3]:
# 假设词汇表的大小为3， 语料包含两个单词"2 0"
word_labels = tf.constant([2, 0])

# 假设模型对两个单词预测时，产生的logit分别是[2.0, -1.0, 3.0]和[1.0, 0.0, -0.5]
predict_logits = tf.constant([[2.0, -1.0, 3.0], [1.0, 0.0, -0.5]])

# 使用sparse_softmax_cross_entropy_with_logits计算交叉熵。
loss = tf.nn.sparse_softmax_cross_entropy_with_logits(
    labels=word_labels, logits=predict_logits)

# 运行程序，计算loss的结果是[0.32656264, 0.46436879], 这对应两个预测的
# perplexity损失。
sess = tf.Session()
print(sess.run(loss))

[0.32656264 0.4643688 ]


### softmax_cross_entropy_with_logits

In [4]:
# softmax_cross_entropy_with_logits与上面的函数相似，但是需要将预测目标以
# 概率分布的形式给出。
word_prob_distribution = tf.constant([[0.0, 0.0, 1.0], [1.0, 0.0, 0.0]])
loss = tf.nn.softmax_cross_entropy_with_logits(
    labels=word_prob_distribution, logits=predict_logits)
# 运行结果与上面相同：[ 0.32656264,  0.46436879]
print(sess.run(loss))

# label smoothing：将正确数据的概率设为一个比1.0略小的值，将错误数据的概率
# 设为比0.0略大的值，这样可以避免模型与数据过拟合，在某些时候可以提高训练效果。
word_prob_smooth = tf.constant([[0.01, 0.01, 0.98], [0.98, 0.01, 0.01]])
loss = tf.nn.softmax_cross_entropy_with_logits(
    labels=word_prob_smooth, logits=predict_logits)
# 运行结果：[ 0.37656265,  0.48936883]
print(sess.run(loss))

sess.close()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

[0.32656264 0.4643688 ]
[0.37656265 0.48936883]


In [6]:
loss2 = tf.nn.softmax_cross_entropy_with_logits_v2(labels=word_prob_smooth, logits=predict_logits)

In [7]:
sess = tf.Session()
print(sess.run(loss))

sess.close()

[0.37656265 0.48936883]
